#### Importações

In [51]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [52]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [53]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [54]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [55]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,122.320300,22.307377,18.087184,99.000,99.000,107.500,1.070000e+02,1.160000e+02,1.170000e+02,142.500000,138.000000,1.560000e+02,1.550000e+02
weight (t),18815.714286,17046.900400,5472.264453,4821.527496,10054.000,10054.000,16530.500,1.263875e+04,1.858400e+04,1.687600e+04,21182.000000,20983.000000,2.764700e+04,2.756900e+04
installation_date,38.857143,36.312900,5.639993,5.456626,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.400000e+01,42.500000,42.000000,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.377200,0.786796,0.728542,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.465200,1.511858,1.945242,4.000,4.000,8.000,4.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
number_of_piles,17.142857,17.522300,10.106575,9.168670,5.000,5.000,8.500,8.000000e+00,2.000000e+01,1.800000e+01,23.000000,26.000000,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,138.810120,18.993420,11.277906,114.000,114.608,122.750,1.301298e+02,1.360000e+02,1.390960e+02,150.500000,147.331250,1.660000e+02,1.655530e+02
distance_to_coast (km),218.571429,234.433200,66.271807,61.704276,120.000,120.000,180.000,1.820000e+02,2.400000e+02,2.750000e+02,264.000000,281.000000,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.561100,0.534522,0.496278,0.000,0.000,0.000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000005,0.000009,0.000008,0.000,0.000,0.000,1.494288e-10,7.800000e-08,2.283485e-08,0.000012,0.000004,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [56]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [57]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [58]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [59]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [60]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [61]:
df_gan_limpo = remover_outliers_iqr(df_gan)

4417


In [62]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (5583, 25))

In [63]:
df_gan_limpo.duplicated().sum()

0

#### Comparando novamente as bases de dados

In [64]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [65]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,1.121497e+02,22.307377,9.754555e+00,99.000,99.000,107.500,1.040000e+02,1.160000e+02,1.100000e+02,142.500000,1.180000e+02,1.560000e+02,1.500000e+02
weight (t),18815.714286,1.468365e+04,5472.264453,4.128509e+03,10054.000,10054.000,16530.500,1.105850e+04,1.858400e+04,1.349300e+04,21182.000000,1.764450e+04,2.764700e+04,2.686300e+04
installation_date,38.857143,3.375837e+01,5.639993,3.831385e+00,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.200000e+01,42.500000,3.600000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.000000e+00,0.786796,0.000000e+00,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000e+00,3.000000e+00,1.000000e+00
number_of_legs,7.428571,5.569765e+00,1.511858,1.953351e+00,4.000,4.000,8.000,4.000000e+00,8.000000e+00,4.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,1.118807e+01,10.106575,6.267856e+00,5.000,5.000,8.500,6.000000e+00,2.000000e+01,9.000000e+00,23.000000,1.500000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,1.356620e+02,18.993420,1.088773e+01,114.000,114.608,122.750,1.268155e+02,1.360000e+02,1.351270e+02,150.500000,1.441705e+02,1.660000e+02,1.639060e+02
distance_to_coast (km),218.571429,2.686135e+02,66.271807,2.871827e+01,120.000,123.000,180.000,2.740000e+02,2.400000e+02,2.810000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,9.222640e-01,0.534522,2.677797e-01,0.000,0.000,0.000,1.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,2.319129e-07,0.000009,9.159946e-07,0.000,0.000,0.000,9.596348e-12,7.800000e-08,6.162226e-10,0.000012,2.687937e-08,2.300000e-05,1.103043e-05


In [66]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),2.108271,4.220193e+00,0.000,5.000000e-01,-1.000000e+00,4.500000,1.000
weight (t),1768.813886,6.507370e+02,0.000,3.891750e+03,1.708000e+03,199.000000,78.000
installation_date,2.544243,1.833669e-01,0.000,3.500000e+00,7.000000e+00,0.500000,0.000
type_of_production (1 oil and gas; 2 oil; 3 gas),0.194229,5.825394e-02,0.000,0.000000e+00,0.000000e+00,1.000000,0.000
number_of_legs,0.963371,-4.333843e-01,0.000,4.000000e+00,0.000000e+00,0.000000,0.000
number_of_piles,-0.379443,9.379050e-01,0.000,5.000000e-01,2.000000e+00,-3.000000,1.000
height_of_jacket_or_sub-structure (m),-1.310120,7.715514e+00,-0.608,-7.379750e+00,-3.096000e+00,3.168750,0.447
distance_to_coast (km),-15.861771,4.567531e+00,0.000,-2.000000e+00,-3.500000e+01,-17.000000,0.000
risk_to_other_users-complete,-0.132529,3.824492e-02,0.000,0.000000e+00,-1.000000e+00,0.000000,0.000
risk_to_other_users-partial,0.000002,9.863048e-07,0.000,-1.494288e-10,5.516515e-08,0.000007,0.000


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [67]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [68]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [69]:
df_gan_limpo.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,105,20869,38,1,8,15,120.049,278,1,2.401143e-09,0.295,0.134,1134187,860060,89030,62573,0.000,0.000,0.995,0.946,0.889,0.593,0.730,1.000,0.439
2,108,12805,31,1,4,11,139.998,281,1,4.023612e-10,0.144,0.092,801732,466598,46870,36178,0.000,0.001,0.990,0.916,0.788,0.392,0.670,1.000,0.425
5,113,20762,37,1,8,24,130.765,278,1,2.637556e-07,0.102,0.044,923674,542459,52760,47879,0.198,0.001,1.000,0.999,0.912,0.900,0.912,0.999,0.428
6,101,24909,35,1,8,12,117.129,281,1,4.866719e-11,0.303,0.134,1160473,877029,91479,63520,0.000,0.000,0.998,0.936,0.890,0.662,0.674,1.000,0.426
9,117,10623,31,1,4,7,148.223,281,1,1.026809e-09,0.063,0.053,354142,433294,24705,31119,0.005,0.001,0.999,0.996,0.683,0.553,0.487,1.000,0.411


In [70]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# Criar o metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=A)
metadata_dict = metadata.to_dict()

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 695.79it/s]|
Column Shapes Score: 71.46%

(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:00<00:00, 356.15it/s]|
Column Pair Trends Score: 92.59%

Overall Score (Average): 82.03%



In [71]:
report.get_score()

0.8202597001208229

In [72]:
report.get_properties()

,Property,Score
0,Column Shapes,0.714603
1,Column Pair Trends,0.925916


In [73]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,installation_date,KSComplement,0.722471
1,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.800729
2,number_of_legs,KSComplement,0.759157
3,number_of_piles,KSComplement,0.857143
4,height_of_jacket_or_sub-structure (m),KSComplement,0.666129
5,risk_to_other_users-complete,KSComplement,0.867471
6,risk_to_other_users-partial,KSComplement,0.641729
7,risk_to_personnel-complete,KSComplement,0.763686
8,risk_to_personnel-partial,KSComplement,0.744429
9,impacts_of_option-complete,KSComplement,0.769914


In [74]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.955277,0.622398,0.532952
1,installation_date,number_of_legs,CorrelationSimilarity,0.959578,0.614305,0.533461
2,installation_date,number_of_piles,CorrelationSimilarity,0.969259,0.681692,0.620209
3,installation_date,height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.714143,0.599779,0.028064
4,installation_date,risk_to_other_users-complete,CorrelationSimilarity,0.933974,-0.473868,-0.605920
5,installation_date,risk_to_other_users-partial,CorrelationSimilarity,0.986197,0.826764,0.799159
6,installation_date,risk_to_personnel-complete,CorrelationSimilarity,0.972961,-0.277817,-0.223738
7,installation_date,risk_to_personnel-partial,CorrelationSimilarity,0.954256,-0.274356,-0.365844
8,installation_date,impacts_of_option-complete,CorrelationSimilarity,0.970976,0.698996,0.640948
9,installation_date,technical_feasibility_or_challenge-complete,CorrelationSimilarity,0.997232,0.397458,0.402994
